## MBenes

In [ ]:
from ase.spacegroup import crystal
from ase.visualize import view

a, b, c = 2.9343, 18.8911, 2.9733
spacegroup = 71

atoms = [
    ('Cr', [0, 0.2936, 0]),   # Cr1 at 4g
    ('Cr', [0.5, 0.5859, 0]), # Cr2 at 4h
    ('Al', [0, 0.5, 0.5]),    # Al at 2b
    ('B', [0, 0.3839, 0.5]),  # B1 at 4h
    ('B', [0.5, 0.6646, 0.5]) # B2 at 4g
]

crystal_structure = crystal(
    symbols=[atom[0] for atom in atoms],
    basis=[atom[1] for atom in atoms],  
    spacegroup=spacegroup,
    cellpar=[a, b, c, 90, 90, 90]       
)
view(crystal_structure)

MAB phases possess variable chemical formulas with the relevant compositions being MAB, M2AB2, M3AB4, and M4AB6, for which M represents an early transition metal and A stands for an IIIA and IVA group element.[6] When assuming A = Al for the sake of simplicity, MAlB phases exhibit an orthorhombic crystal structure with space groups varying from Cmcm (for MAlB) over Cmmm (for M2AlB2 and M4AlB6) to Pmmm (for M3AlB4).[6]

For MBenes, it is important to emphasize that the M-B layers are separated by either mono- and bi-layers of Al.

The crystal structures of various MAB phases and related MBenes (Figure 2a) can be classified in the following manner. The first group of compounds are “212” phases with Cmmm crystal symmetry, “414” phases with Immm symmetry, and “222” with Cmcm symmetry.[3] The second group consists of “314” structures with Pmmm symmetry, whereas the third group is formed by “416” structures with Cmmm symmetry.



In [ ]:
from ase import Atom, Atoms
from ase.build import mx2, molecule
from ase.data import chemical_symbols
from ase.io import read, write
from ase.visualize import view
from itertools import combinations
import numpy as np
import random

class MBeneBuilder:
    """
    用于搭建高对称的单过渡金属 MBene 2D结构的 python 类。
    """
    def __init__(self, mab_type="212", m_element="Cr", a_element="Al", b_element="B", 
                 lattice_a=0.0, lattice_b=0.0, lattice_c=0.0, 
                 supercell_matrix=(3, 3, 1), vacuum=15.0, n_layers=1, layer_distance=5.0, **kwargs):
        self.mab_type = mab_type
        self.m_element = m_element
        self.a_element = a_element
        self.b_element = b_element
        self.lattice_a = lattice_a
        self.lattice_b = lattice_b
        self.lattice_c = lattice_c
        
        self.supercell_matrix = supercell_matrix
        self.vacuum = vacuum
        self.n_layers = n_layers
        self.layer_distance = layer_distance
        
    def predefined_systems(self):
        predefined_mab_dict = {
            "212": {
                "symbols": [self.m_element] * 4 + [self.a_element] * 2 + [self.b_element] * 4,
                "cell": [2.908355, 2.909313, 10.985405],
                "scaled_positions": [
                    [0.500000, 0.500000, 0.851860],
                    [0.000000, 0.500000, 0.648140],
                    [0.000000, 0.500000, 0.351860],
                    [0.500000, 0.500000, 0.148140],
                    [0.000000, 0.000000, 0.000000], # Al1
                    [0.500000, 0.000000, 0.500000], # Al2
                    [0.500000, 0.000000, 0.705053],
                    [0.000000, 0.000000, 0.794947],
                    [0.000000, 0.000000, 0.205053],
                    [0.500000, 0.000000, 0.294947],
                ],
            },
            "222": {
                "symbols": [self.m_element] * 4 + [self.a_element] * 4 + [self.b_element] * 4,
                "cell": [3.205721, 3.096498, 13.969478],
                "scaled_positions": [
                    [0.500000, 0.250000, 0.410469], 
                    [0.000000, 0.750000, 0.089531], 
                    [0.000000, 0.250000, 0.910469], 
                    [0.500000, 0.750000, 0.589531], 
                    [0.000000, 0.750000, 0.301286], # Al1
                    [0.500000, 0.250000, 0.198714], # Al2
                    [0.500000, 0.750000, 0.801286], # Al3
                    [0.000000, 0.250000, 0.698714], # Al4
                    [0.500000, 0.250000, 0.033641], 
                    [0.000000, 0.750000, 0.466359], 
                    [0.000000, 0.250000, 0.533641], 
                    [0.500000, 0.750000, 0.966359], 
                ],
            },
            "314": {
                "symbols": [self.m_element] * 3 + [self.a_element] * 1 + [self.b_element] * 4,
                "cell": [2.918412, 2.920923, 8.058275],
                "scaled_positions": [
                    [0.000000, 0.000000, 0.702008],
                    [0.000000, 0.000000, 0.297992],
                    [0.500000, 0.000000, 0.000000],
                    [0.500000, 0.500000, 0.500000], # Al1
                    [0.000000, 0.500000, 0.108009],
                    [0.500000, 0.500000, 0.223790],
                    [0.000000, 0.500000, 0.891991],
                    [0.500000, 0.500000, 0.776210],
                ],
            },
            "322": {
                "symbols": [self.m_element] * 6 + [self.a_element] * 4 + [self.b_element] * 4,
                "cell": [3.038952, 2.942621, 17.826332],
                "scaled_positions": [
                    [0.500000, 0.000000, 0.325048], 
                    [0.000000, 0.000000, 0.000000], 
                    [0.000000, 0.000000, 0.174952], 
                    [0.000000, 0.000000, 0.825048], 
                    [0.500000, 0.000000, 0.500000], 
                    [0.500000, 0.000000, 0.674952], 
                    [0.500000, 0.500000, 0.079035], # Al1
                    [0.000000, 0.500000, 0.420965], # Al2
                    [0.000000, 0.500000, 0.579035], # Al3
                    [0.500000, 0.500000, 0.920965], # Al4
                    [0.500000, 0.500000, 0.222358], 
                    [0.000000, 0.500000, 0.277642], 
                    [0.000000, 0.500000, 0.722358], 
                    [0.500000, 0.500000, 0.777642], 
                ],
            },
            "414": {
                "symbols": [self.m_element] * 8 + [self.a_element] * 2 + [self.b_element] * 8,
                "cell": [2.934300, 2.973300, 18.891100],
                "scaled_positions": [
                    [0.500000, 0.500000, 0.206400],
                    [0.000000, 0.000000, 0.293600],
                    [0.500000, 0.000000, 0.414100],
                    [0.000000, 0.500000, 0.085900],
                    [0.000000, 0.000000, 0.706400],
                    [0.500000, 0.500000, 0.793600],
                    [0.000000, 0.500000, 0.914100],
                    [0.500000, 0.000000, 0.585900],
                    [0.500000, 0.000000, 0.000000], # Al1
                    [0.000000, 0.500000, 0.500000], # Al2
                    [0.500000, 0.000000, 0.116100],
                    [0.000000, 0.500000, 0.383900],
                    [0.500000, 0.500000, 0.335400],
                    [0.000000, 0.000000, 0.164600],
                    [0.000000, 0.500000, 0.616100],
                    [0.500000, 0.000000, 0.883900],
                    [0.000000, 0.000000, 0.835400],
                    [0.500000, 0.500000, 0.664600],
                ],
            },
            "416": {
                "symbols": [self.m_element] * 8 + [self.a_element] * 2 + [self.b_element] * 12,
                "cell": [2.951706, 3.012993, 21.280000],
                "scaled_positions": [
                    [0.000000, 0.500000, 0.576340], 
                    [0.500000, 0.500000, 0.923660], 
                    [0.500000, 0.500000, 0.688430], 
                    [0.000000, 0.500000, 0.811570], 
                    [0.500000, 0.500000, 0.076340], 
                    [0.000000, 0.500000, 0.423660], 
                    [0.000000, 0.500000, 0.188430], 
                    [0.500000, 0.500000, 0.311570], 
                    [0.000000, 0.000000, 0.000000], # Al1
                    [0.500000, 0.000000, 0.500000], # Al2
                    [0.500000, 0.000000, 0.605000], 
                    [0.000000, 0.000000, 0.895000], 
                    [0.000000, 0.000000, 0.648100],
                    [0.500000, 0.000000, 0.851900], 
                    [0.000000, 0.000000, 0.729800], 
                    [0.500000, 0.000000, 0.770200], 
                    [0.000000, 0.000000, 0.105000], 
                    [0.500000, 0.000000, 0.395000], 
                    [0.500000, 0.000000, 0.148100],
                    [0.000000, 0.000000, 0.351900], 
                    [0.500000, 0.000000, 0.229800], 
                    [0.000000, 0.000000, 0.270200], 
                ],
            },
        }
        return predefined_mab_dict
    
    def _generate_mab_structure(self):
        mab_dict = self.predefined_systems()[self.mab_type]
        cell = [
            self.lattice_a if self.lattice_a != 0 else mab_dict["cell"][0],
            self.lattice_b if self.lattice_b != 0 else mab_dict["cell"][1],
            self.lattice_c if self.lattice_c != 0 else mab_dict["cell"][2]
        ]
        return Atoms(symbols=mab_dict["symbols"], 
                     scaled_positions=mab_dict["scaled_positions"], 
                     cell=cell, pbc=True)
    
    def _filter_a_elements(self, structure, first_a_z, second_a_z):
        return structure[[atom.index for atom in structure if atom.symbol != self.a_element and not (atom.position[2] < first_a_z or atom.position[2] > second_a_z)]]

    def _generate_single_mbene_structure(self):
        structure = self._generate_mab_structure().copy() * (1, 1, 2)
        a_element_z_coords = [atom.position[2] for atom in structure if atom.symbol == self.a_element]
        first_a_element_z, second_a_element_z = sorted(set(a_element_z_coords))[:2]
        
        structure = self._filter_a_elements(structure, first_a_element_z, second_a_element_z)
        structure.center()
        return structure
    
    def _generate_mbene_structure(self):
        if self.supercell_matrix[2] != 1:
            print(f"Warning: The third component of supercell_matrix should be 1, but got {self.supercell_matrix[2]}. It will be set to 1.")
            self.supercell_matrix = (self.supercell_matrix[0], self.supercell_matrix[1], 1)

        structure = self._generate_single_mbene_structure().copy() * self.supercell_matrix

        if self.n_layers > 1:
            single_layer_height = max(atom.position[2] for atom in structure) - min(atom.position[2] for atom in structure)
            for layer in range(1, self.n_layers):
                new_layer = structure.copy()
                for atom in new_layer:
                    atom.position[2] += layer * (single_layer_height + self.layer_distance)
                structure += new_layer
                
        z_length_with_vacuum = (max(atom.position[2] for atom in structure) - min(atom.position[2] for atom in structure)) + self.vacuum
        structure.set_cell([structure.cell[0], structure.cell[1], [0, 0, z_length_with_vacuum]])
        structure.center()
        
        return structure

In [ ]:
system = MBeneBuilder(mab_type="314")
atoms = system._generate_mbene_structure()
view(atoms)
# write("mxene.vasp", atoms, sort=True, vasp5=True)

### Q-Api 接口

In [ ]:
def mbene_builder(**kwargs):
    kwargs['supercell_matrix'] = (kwargs['n_a'], kwargs['n_b'], 1)
    system = MBeneBuilder(**kwargs)
    new_structure = system._generate_mbene_structure()
    cif_output = io.BytesIO()
    write(cif_output, new_structure, format='cif')
    cif_string = cif_output.getvalue().decode('utf-8')
    return cif_string

In [ ]:
@guide_register_func(ModelTag('构建高对称 MBene 材料结构', ['催化','电池'],['B'],nano_type='2d').identifier)
def MBene(data):
    mode = data.get('mode')
    if mode == 'init':
        title = Description(name='title', note='构建高对称 MBene 材料结构，需要指定 MAB 相基体元素及晶格参数等信息')
        mab_type = SingleFromList(name='mab_type', note='MAB 相类型', id='mab_type', default_value='212', list_value=[{'label': str(i), 'value': str(i)} for i in [212, 222, 314, 322, 414, 416]], is_required=1)
        m_element = SingleFromList(name='m_element', 
                                   note='MAB 相基体中的 M 元素', 
                                   id='m_element', 
                                   list_value=[{'label': '钪', 'value': 'Sc'},
                                               {'label': '钛', 'value': 'Ti'},
                                               {'label': '锆', 'value': 'Zr'},
                                               {'label': '铪', 'value': 'Hf'},
                                               {'label': '钒', 'value': 'V' },
                                               {'label': '铌', 'value': 'Nb'},
                                               {'label': '钽', 'value': 'Ta'},
                                               {'label': '铬', 'value': 'Cr'},
                                               {'label': '钼', 'value': 'Mo'},
                                               {'label': '钨', 'value': 'W' },
                                               {'label': '锰', 'value': 'Mn' },
                                               {'label': '锝', 'value': 'Tc' },
                                               {'label': '铁', 'value': 'Fe' },
                                               {'label': '钌', 'value': 'Ru' },
                                               {'label': '铑', 'value': 'Rh' },
                                               {'label': '镍', 'value': 'Ni' }],
                                   is_required=1)
        a_element = SingleFromList(name='a_element', 
                                   note='MAB 相基体中的 A 元素', 
                                   id='a_element', 
                                   list_value=[{'label': '铝', 'value': 'Al'},
                                               {'label': '硅', 'value': 'Si'},
                                               {'label': '镓', 'value': 'Ga'},
                                               {'label': '锗', 'value': 'Ge'},
                                               {'label': '锡', 'value': 'Sn'}],
                                   is_required=1)
        b_element = SingleFromList(name='b_element', 
                                   note='MAB 相基体中的 B 元素', 
                                   id='b_element', 
                                   list_value=[{'label': '硼', 'value': 'B'}],
                                   is_required=1)
        lattice_a = SingleInput(name='lattice_a', note='a 方向晶格常数 (Å)', id='lattice_a', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        lattice_b = SingleInput(name='lattice_b', note='b 方向晶格常数 (Å)', id='lattice_b', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        lattice_c = SingleInput(name='lattice_c', note='c 方向晶格常数 (Å)', id='lattice_c', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        n_a = SingleInput(name='n_a', note='a 轴的单胞重复单元', id='n_a', input_type='int', default_value=3, min=1, max=10, is_required=0)
        n_b = SingleInput(name='n_b', note='b 轴的单胞重复单元', id='n_b', input_type='int', default_value=3, min=1, max=10, is_required=0)
        vacuum = SingleInput(name='vacuum', note='真空层厚度 (Å)', id='vacuum', input_type='float', default_value=15.0, min=10.0, max=100.0, is_required=0)
        n_layers = SingleInput(name='n_layers', note='MBene 重复单元数', id='n_layers', input_type='int', default_value=1, min=1, max=10, is_required=0)
        layer_distance = SingleInput(name='layer_distance', note='MBene 层间距 (Å)', id='layer_distance', input_type='float', default_value=5.0, min=3.0, max=100.0, is_required=0)
        
        return [title(), mab_type(), m_element(), a_element(), b_element(), lattice_a(), lattice_b(), lattice_c(), n_a(), n_b(), vacuum(), n_layers(), layer_distance()]
        
    elif mode == 'generate':
        value = data.get('value')
        cif_string = mbene_builder(**value)
        return {'file_content': cif_string, 'file_format': 'cif'}
